# **Replicating LeNet and AlexNet in Tensorflow2.0 using Keras**

---

In this lesson, we use **Keras with a TensorFlow 2.0** Backend to to replicate both **LeNet and AlexNet** in Keras and train it to **recognize handwritten digits in the MNIST dataset and the 10 images classes of CIFAR10**
1. Replicate the LeNet CNN Architecture
2. Replicate the AlexNet CNN Architecture

## **Let's construct LeNet in Keras!**

![](https://www.researchgate.net/profile/Sheraz_Khan8/publication/321586653/figure/fig4/AS:568546847014912@1512563539828/The-LeNet-5-Architecture-a-convolutional-neural-network.png)
## **LeNet Architecture**
S.No | Layers | Output Shape (Height, Width, Channels)
--- | --- | ---
1 | Input Layer | 32 x 32 x 1
2 | Conv2d [6 Filters of size = 5x5, stride = 1, padding = 0 ] | 28 x 28 x 6
3 | Average Pooling [stride = 2, padding = 0] | 14 x 14 x 6
4 | Conv2d [16 Filters of size = 5x5, stride = 1, padding = 0 ] | 10 x 10 x 16
5 | Average Pooling [stride = 2, padding = 0] | 5 x 5 x 16
6 | Conv2d [120 Filters of size = 5x5, stride = 1, padding = 0 ] | 1 x 1 x 120
7 | Linear1 Layer | 120
8 | Linear2 Layer | 84
9 | Final Linear Layer | 10


### **Loading and preprocessing our Data**

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adadelta

# loads the MNIST dataset
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

# Lets store the number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[1].shape[0]

# Getting our date in the right 'shape' needed for Keras
# We need to add a 4th dimenion to our date thereby changing our
# Our original image shape of (60000,28,28) to (60000,28,28,1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# store the shape of a single image
input_shape = (img_rows, img_cols, 1)

# change our image type to float32 data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize our data by changing the range from (0 to 255) to (0 to 1)
x_train /= 255
x_test /= 255

# Now we one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

11490434/11490434 [==============================] - 0s 0us/step


### **Now let's create our layers to replicate LeNet**

In [3]:
# create model
model = Sequential()

# 2 sets of CRP (Convolution, RELU, Pooling)
model.add(Conv2D(6, (5, 5),
                 padding = "same", # padding is set to 0 in the original paper, to set padding at 0, use "valid"
                 input_shape = input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

model.add(Conv2D(16, (5, 5),
                 padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

model.add(Conv2D(120, (5, 5),
                 padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))

# Fully connected layers (w/ RELU)
model.add(Flatten())
model.add(Dense(120))
model.add(Activation("relu"))

model.add(Dense(84))
model.add(Activation("relu"))
# Softmax (for classification)
model.add(Dense(num_classes))
model.add(Activation("softmax"))

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 activation (Activation)     (None, 28, 28, 6)         0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 6)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        2416      
                                                                 
 activation_1 (Activation)   (None, 14, 14, 16)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 16)          0         
 g2D)                                                   

### **Now let us train LeNet on our MNIST Dataset**

In [ ]:
# Training Parameters
batch_size = 128
epochs = 50

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

model.save("mnist_LeNet.h5")

# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/50
469/469 [==============================] - 16s 7ms/step - loss: 2.3027 - accuracy: 0.1167 - val_loss: 2.2963 - val_accuracy: 0.1347
Epoch 2/50
469/469 [==============================] - 3s 6ms/step - loss: 2.2902 - accuracy: 0.1597 - val_loss: 2.2835 - val_accuracy: 0.2096
Epoch 3/50
469/469 [==============================] - 3s 6ms/step - loss: 2.2777 - accuracy: 0.2660 - val_loss: 2.2704 - val_accuracy: 0.3153
Epoch 4/50
469/469 [==============================] - 3s 7ms/step - loss: 2.2646 - accuracy: 0.3360 - val_loss: 2.2564 - val_accuracy: 0.3668
Epoch 5/50
469/469 [==============================] - 3s 7ms/step - loss: 2.2496 - accuracy: 0.3776 - val_loss: 2.2394 - val_accuracy: 0.4139
Epoch 6/50
469/469 [==============================] - 3s 6ms/step - loss: 2.2308 - accuracy: 0.4229 - val_loss: 2.2174 - val_accuracy: 0.4498
Epoch 7/50
469/469 [==============================] - 3s 6ms/step - loss: 2.2059 - accuracy: 0.4622 - val_loss: 2.1884 - val_accuracy: 0.4888
Epoch

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - 1s 3ms/step - loss: 0.2933 - accuracy: 0.9195
Test loss: 0.29328733682632446
Test accuracy: 0.9194999933242798


## **Now let's replicate AlexNET and train in on the CIFAR10 Dataset**

AlexNet was the 2012 ImageNet winner achieved a top-5 error of 15.3%, more than 10.8 percentage points lower than that of the runner up!

![](https://paperswithcode.com/media/methods/Screen_Shot_2020-06-22_at_6.35.45_PM.png)

![](https://production-media.paperswithcode.com/datasets/CIFAR-10-0000000431-b71f61c0_U5n3Glr.jpg)

In [4]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.utils import to_categorical

# Loads the CIFAR dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Display our data shape/dimensions
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Now we one hot encode outputs
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

170498071/170498071 [==============================] - 5s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [8]:
l2_reg = 0.001

# Initialize model
model = Sequential()

# 1st Conv Layer
model.add(Conv2D(96, (11, 11), input_shape=x_train.shape[1:],
    padding='same', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Conv Layer
model.add(Conv2D(256, (5, 5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Conv Layer
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 4th Conv Layer
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

# 5th Conv Layer
model.add(ZeroPadding2D((1, 1)))
model.add(Conv2D(1024, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 1st FC Layer
model.add(Flatten())
model.add(Dense(3072))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 2nd FC Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 3rd FC Layer
model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

print(model.summary())

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              metrics = ['accuracy'])


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 32, 32, 96)        34944     
                                                                 
 batch_normalization_24 (Ba  (None, 32, 32, 96)        384       
 tchNormalization)                                               
                                                                 
 activation_30 (Activation)  (None, 32, 32, 96)        0         
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 16, 16, 96)        0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 16, 16, 256)       614656    
                                                                 
 batch_normalization_25 (Ba  (None, 16, 16, 256)      

In [ ]:
# Training Parameters
batch_size = 64
epochs = 25

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

model.save("CIFAR10_AlexNet_10_Epoch.h5")

# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/25
782/782 [==============================] - 97s 112ms/step - loss: 2.1335 - accuracy: 0.2457 - val_loss: 1.7633 - val_accuracy: 0.3905
Epoch 2/25
782/782 [==============================] - 85s 109ms/step - loss: 1.8479 - accuracy: 0.3399 - val_loss: 1.6670 - val_accuracy: 0.4297
Epoch 3/25
782/782 [==============================] - 85s 109ms/step - loss: 1.7451 - accuracy: 0.3867 - val_loss: 1.6035 - val_accuracy: 0.4571
Epoch 4/25
782/782 [==============================] - 85s 109ms/step - loss: 1.6836 - accuracy: 0.4130 - val_loss: 1.5598 - val_accuracy: 0.4770
Epoch 5/25
782/782 [==============================] - 85s 109ms/step - loss: 1.6334 - accuracy: 0.4366 - val_loss: 1.5260 - val_accuracy: 0.4883
Epoch 6/25
782/782 [==============================] - 85s 109ms/step - loss: 1.5943 - accuracy: 0.4518 - val_loss: 1.4972 - val_accuracy: 0.4980
Epoch 7/25
782/782 [==============================] - 85s 109ms/step - loss: 1.5623 - accuracy: 0.4699 - val_loss: 1.4698 - val_ac

## **Current Top Performers in CIFAR10**
https://paperswithcode.com/sota/image-classification-on-cifar-10